In [1]:
import os
import pandas as pd
import shutil
import re
import pprint
import math
import numpy as np

from ucsbdistrict.inputs import CUP_inputs, weather_df, heatPump_df,districtModule_inputs,TES_inputs,dateTime_df,coldTank_schedule2_inputs_df,hotTank_schedule1_inputs_df 


inputs 1268952.1046379893


In [2]:
# Load the DataFrame from the pickle file
buildingMixing_output = pd.read_pickle('buildingMixing_output.pkl')

# Now 'df' contains the DataFrame loaded from the pickle file
buildingMixing_output.head()  # Verify the DataFrame

,Total Space Heating Load (Btu/h),Total DHW Load (Btu/h),Total Heating Load (Btu/h),District HWS Flow (gpm),District HWRT (°F),District CHWS Flow (gpm),District CHWRT (°F),Total Cooling Load (Btu/h)
0,1.231706e+07,3.827353e+06,1.614441e+07,520.917695,103.0155,0.024464,54.000009,122.322571
1,9.027328e+06,3.236478e+06,1.226381e+07,389.913673,102.094763,0.174641,54.000061,873.211983
2,1.029800e+07,3.468767e+06,1.376677e+07,440.544528,102.501126,0.174641,54.000061,873.211983
3,1.133739e+07,3.165227e+06,1.450262e+07,472.558806,103.620896,0.174641,54.000061,873.211983
4,1.177647e+07,4.181198e+06,1.595766e+07,508.396115,102.223504,0.174641,54.000061,873.211983


In [3]:
##Heat pump input

districtHWRT = buildingMixing_output["District HWRT (°F)"]
districtCHWRT = buildingMixing_output["District CHWRT (°F)"]
districtHWSflow = buildingMixing_output["District HWS Flow (gpm)"]
districtCHWSflow = buildingMixing_output["District CHWS Flow (gpm)"]
HP_leavingHW = heatPump_df["Leaving Hot Water (°F)"]
HP_enteringHW = heatPump_df["Entering Hot Water (°F)"]
HP_enteringCHW = heatPump_df["Entering Cold Water (°F)"]
HP_leavingCHW = heatPump_df["Leaving Cold Water (°F)"]
HP_maxHeating = heatPump_df["Max Heating Output (Btu/h)"]
HP_maxCooling = heatPump_df["Max Cooling Output (Btu/h)"]
HW_districtSTP =  districtModule_inputs["Loop HW STP (°F)"]
HP_heatingCapacity=CUP_inputs["Heat Pumps"]["Heating Capacity"]
HP_coolingCapacity=CUP_inputs["Heat Pumps"]["Cooling Capacity"]
HP_waterSource=CUP_inputs["Heat Pumps"]["Water-Source Only %"]
HP_airWaterSource=CUP_inputs["Heat Pumps"]["Air-Source + Water Source %"]





In [4]:
from ucsbdistrict.heatPump import heatPump

heatPump_instance = heatPump(districtHWRT=districtHWRT,districtCHWRT=districtCHWRT,HW_districtSTP = HW_districtSTP,\
                             HP_leavingHW=HP_leavingHW,HP_enteringHW=HP_enteringHW,HP_enteringCHW=HP_enteringCHW,HP_leavingCHW=HP_leavingCHW,\
                              HP_maxHeating=HP_maxHeating,HP_maxCooling=HP_maxCooling )

In [5]:
heatPump_instance.compute()

ppp 8    96.1
8    96.1
8    96.1
8    96.1
8    96.1
     ... 
8    96.1
8    96.1
8    96.1
8    96.1
8    96.1
Name: Max Cooling Output (Btu/h), Length: 8760, dtype: float64


,HP CHWST (°F),Max Unitary Heating Output (Btu/h),Max Unitary Cooling Output (Btu/h),Heat:Cool
0,44,145.6,96.1,1.515088
1,44,145.6,96.1,1.515088
2,44,145.6,96.1,1.515088
3,44,145.6,96.1,1.515088
4,44,145.6,96.1,1.515088
...,...,...,...,...
8755,44,145.6,96.1,1.515088
8756,44,145.6,96.1,1.515088
8757,44,145.6,96.1,1.515088
8758,44,145.6,96.1,1.515088


In [6]:
print(HP_enteringHW - districtHWRT)

0        -23.0155
1      -22.094763
2      -22.501126
3      -23.620896
4      -22.223504
          ...    
8755          NaN
8756          NaN
8757          NaN
8758          NaN
8759          NaN
Length: 8760, dtype: object


In [7]:
#TES inputs

HW_upperTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Upper Tank Volume"]
HW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Hot Tank Lower Tank Volume"]
HW_TankVol = TES_inputs["Tank Properties"]["TES Hot Tank Volume"]
HW_wallLoss = TES_inputs["Tank Properties"]["TES Hot Wall-Losses"]
HW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Hot Thermocline/other-Losses"]
CHW_upperTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Upper Tank Volume"]
CHW_lowerTankVol = TES_inputs["Initial Conditions"]["TES Cold Tank Lower Tank Volume"]
CHW_TankVol = TES_inputs["Tank Properties"]["TES Cold Tank Volume"]
CHW_wallLoss = TES_inputs["Tank Properties"]["TES Cold Wall-Losses"]
CHW_ThermocloneLoss = TES_inputs["Tank Properties"]["TES Cold Thermocline/other-Losses"]

#parameters for second instance
"Fraction of Thermocline to discharge"
thermoclineDischarge = TES_inputs["Tank Charging Characteristics"]["Fraction of Thermocline to discharge"]
stopCharging = TES_inputs["Tank Charging Characteristics"]["Stop Charging tank at ____ Capacity"]
totalHeating_load=buildingMixing_output["Total Heating Load (Btu/h)"]
#HP_heatingCapacity taken from heat pump inputs

In [8]:
#TES instance

from ucsbdistrict.TES import TES

TES_instance_one = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
          CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
            HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,totalHeating_load=totalHeating_load)

In [9]:
TES_instance_one.compute()

,check,TES Hot Th after losses (Gal)_supply temp,TES Hot Tc after losses (Gal)_return temp,TES Hot Thermocline (Gal),TES Cold Th after losses (Gal)_supply temp,TES Cold Tc after losses (Gal)_return temp,TES Cold Thermocline (Gal)
0,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
1,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
2,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
3,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
4,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
...,...,...,...,...,...,...,...
8755,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
8756,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
8757,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228
8758,0.957,1.214023e+06,12262.85705,42666.399598,12264.083704,1.214144e+06,42543.734228


In [10]:
#CUP inputs
timeStamp = dateTime_df
hotTank_schedule = hotTank_schedule1_inputs_df 
coldTank_schedule = coldTank_schedule2_inputs_df

totalCooling_load=buildingMixing_output["Total Cooling Load (Btu/h)"]
hotTank_schedule.head()

,Schedules,Load Shift Control,Month,0,1,2,3,4,5,6,...,15,16,17,18,19,20,21,22,23,Total
0,Schedules,Load Shift Control - Hot Tank by month,1,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
1,Schedules,Load Shift Control - Hot Tank by month,2,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
2,Schedules,Load Shift Control - Hot Tank by month,3,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
3,Schedules,Load Shift Control - Hot Tank by month,4,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13
4,Schedules,Load Shift Control - Hot Tank by month,5,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,0,0,13


In [11]:
type(dateTime_df)
# print(timeStamp)
timeStamp.head()

,Date and Time
0,2010-01-01 00:00:00
1,2010-01-01 01:00:00
2,2010-01-01 02:00:00
3,2010-01-01 03:00:00
4,2010-01-01 04:00:00


In [12]:

from ucsbdistrict.CUP import CUP
CUP_instance = CUP(timeStamp=timeStamp,hotTank_schedule=hotTank_schedule,coldTank_schedule=coldTank_schedule,totalHeating_load=totalHeating_load,\
                   totalCooling_load=totalCooling_load)

In [13]:
CUP_instance.compute_CUP()

<class 'pandas.core.series.Series'>
0         0
1         0
2         0
3         0
4         0
       ... 
8755    364
8756    364
8757    364
8758    364
8759    364
Name: shiftCount, Length: 8760, dtype: int32
<class 'pandas.core.series.Series'>
0         0
1         0
2         0
3         0
4         0
       ... 
8755    364
8756    364
8757    364
8758    364
8759    364
Name: shiftCount, Length: 8760, dtype: int32


,Hot Load Shift Charging,Cold Load Shift Charging,Hot Load Shift Count,Predicted Day's Heating Load (Btu/h),Predicted Day's Cooling Load (Btu/h),Predicted Heating Load in Load Shift Window (Btu/h),Predicted Cooling Load in Load Shift Window (Btu/h)
0,False,True,0,4.658689e+08,6.708789e+07,2.770624e+08,5.728561e+03
1,False,True,0,4.658689e+08,6.708789e+07,2.770624e+08,5.728561e+03
2,False,True,0,4.658689e+08,6.708789e+07,2.770624e+08,5.728561e+03
3,False,True,0,4.658689e+08,6.708789e+07,2.770624e+08,5.728561e+03
4,False,True,0,4.658689e+08,6.708789e+07,2.770624e+08,5.728561e+03
...,...,...,...,...,...,...,...
8755,True,False,364,5.032234e+08,2.234066e+08,2.984948e+08,2.170127e+07
8756,True,False,364,5.032234e+08,2.234066e+08,2.984948e+08,2.170127e+07
8757,True,False,364,5.032234e+08,2.234066e+08,2.984948e+08,2.170127e+07
8758,False,True,364,5.032234e+08,2.234066e+08,2.984948e+08,2.170127e+07


In [14]:
# HotLoadShiftCharging = CUP_instance["Hot Load Shift Charging"]

# print(HotLoadShiftCharging)

In [15]:

TES_instance_two = TES(HW_upperTankVol=HW_upperTankVol,HW_TankVol=HW_TankVol,HW_wallLoss=HW_wallLoss,HW_ThermocloneLoss=HW_ThermocloneLoss,HW_lowerTankVol=HW_lowerTankVol,\
                      CHW_upperTankVol=CHW_upperTankVol,CHW_lowerTankVol=CHW_lowerTankVol,CHW_TankVol=CHW_TankVol,CHW_wallLoss=CHW_wallLoss,CHW_ThermocloneLoss=CHW_ThermocloneLoss,\
                      HP_heatingCapacity=HP_heatingCapacity,thermoclineDischarge=thermoclineDischarge,stopCharging=stopCharging,CUP_instance=CUP_instance,totalHeating_load=totalHeating_load)
TES_instance_two.compute_2()


<class 'pandas.core.series.Series'>
0         0
1         0
2         0
3         0
4         0
       ... 
8755    364
8756    364
8757    364
8758    364
8759    364
Name: shiftCount, Length: 8760, dtype: int32
<class 'pandas.core.series.Series'>
0         0
1         0
2         0
3         0
4         0
       ... 
8755    364
8756    364
8757    364
8758    364
8759    364
Name: shiftCount, Length: 8760, dtype: int32


,TES Hot Status
0,Discharge
1,Discharge
2,Discharge
3,Discharge
4,Discharge
...,...
8755,Charge
8756,Charge
8757,Charge
8758,Discharge
